Projet de Machine Learning : expliquer le prix de l'électricité

Romain Foucher
Matthieu Evort
Julie Chen


Introduction :

Le machine learning est une technologie qui vise à apprendre aux machines à tirer des enseignements des données et à s'améliorer avec l'expérience, au lieu d'être explicitement programmées pour le faire.

Ce projet consiste a expliquer le prix de l'électricité en France et en Allemagne selon une multitude de facteurs. On peut citer le climat, le réchauffement climatique, les  événements géopolitiques... De plus, chaque pays s’appuie sur un mix  énergétique qui lui est propre (nucléaire, solaire, hydrolique, gaz, charbon, etc).

 Pour cela, nous allons utiliser plusieurs méthodes afin de préparer nos données, de les analyser et de les modéliser grâce aux cours et aux TP que nous avons pu effectuer en cours mais ainsi qu'à la documantation d'internet.

Sommaire :

A) Préparation des données
B) Analyse exploratoire des données
C) Modélisation des données


A) Préparation des données

Afin de préparer nos données nous allons remplir les cases vides par la moyenne de la colonne. En effet, si nous supprimons les lignes ou les colonnes, nous perdrons des informations et nous réduisons la taille de l'échantillon. Cependant, nous n'allons pas supprimer ou remplacer les données "0.0" car elles sont utiles. En effet, il est possible que la variation du prix du charbon ou du gaz d'un pays ne change pas d'une journée à l'autre.

In [103]:
from Data_preparation import data_x
for col in data_x.columns[data_x.isnull().any()]:
    data_x[col].fillna(data_x[col].mean(), inplace=True)

Ensuite, nous fusionnons data_x et data_y pour comparer nos valeurs d'entrées avec nos valeurs de sorties en joignant sur l'ID. Puis, nous trions sur DAY_ID et COUNTRY pour une meilleure lisibilité.

In [104]:
from Data_preparation import data_x, data_y, pd
merged_X_Y = pd.merge(data_x, data_y, on='ID', how='inner')
merged_X_Y = merged_X_Y.sort_values(by=['DAY_ID', 'COUNTRY'])


Puisque nous étudions la France et l'Allemagne nous créons deux datas : data_FR et data_DE et nous ajoutons leurs colonnes respectives.

In [105]:
data_FR = merged_X_Y.loc[merged_X_Y['COUNTRY'] == 'FR']
data_DE = merged_X_Y.loc[merged_X_Y['COUNTRY'] == 'DE']
data_FR = data_FR.filter(regex='FR|GAS_RET|COAL_RET', axis=1)
data_DE = data_DE.filter(regex='DE|GAS_RET|COAL_RET', axis=1)

Les valeurs des différents attributs sont comparables car ce sont toutes des variations dans le temps.

B) Analyse exploratoire des données

Afin de faire un aperçu des données nous utilisons plusieurs fonctions :


In [106]:
data_x.info()
data_x.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1494 entries, 0 to 1493
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1494 non-null   int64  
 1   DAY_ID            1494 non-null   int64  
 2   COUNTRY           1494 non-null   object 
 3   DE_CONSUMPTION    1494 non-null   float64
 4   FR_CONSUMPTION    1494 non-null   float64
 5   DE_FR_EXCHANGE    1494 non-null   float64
 6   FR_DE_EXCHANGE    1494 non-null   float64
 7   DE_NET_EXPORT     1494 non-null   float64
 8   FR_NET_EXPORT     1494 non-null   float64
 9   DE_NET_IMPORT     1494 non-null   float64
 10  FR_NET_IMPORT     1494 non-null   float64
 11  DE_GAS            1494 non-null   float64
 12  FR_GAS            1494 non-null   float64
 13  DE_COAL           1494 non-null   float64
 14  FR_COAL           1494 non-null   float64
 15  DE_HYDRO          1494 non-null   float64
 16  FR_HYDRO          1494 non-null   float64


,ID,DAY_ID,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,FR_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
count,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,...,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000,1494.000000
mean,1072.759036,591.861446,0.427442,-0.020032,-0.145508,0.145508,-0.256332,-0.072643,0.256332,0.072643,...,-0.153688,-0.037831,0.019357,0.109480,0.123099,0.009451,0.008404,0.058126,0.061724,0.080510
std,618.013179,345.065043,0.673412,0.918995,0.962068,0.962068,0.916821,1.050307,0.916821,1.050307,...,0.896325,0.952745,1.018132,1.022452,1.020950,0.941285,0.971257,1.097768,1.033853,1.098624
min,0.000000,0.000000,-2.265563,-1.462350,-2.856874,-2.634831,-2.464849,-2.825331,-2.279619,-1.951516,...,-1.678936,-2.128531,-1.726420,-1.880419,-1.895319,-4.549638,-5.787097,-5.349463,-5.706442,-4.281790
25%,540.250000,292.250000,-0.037421,-0.716771,-0.869990,-0.631683,-0.921263,-0.820640,-0.395910,-0.774676,...,-0.802333,-0.618791,-0.488625,-0.633867,-0.645796,-0.577139,-0.597690,-0.624238,-0.458038,-0.522968
50%,1077.500000,591.000000,0.357061,-0.394166,-0.145508,0.145508,-0.256332,0.041508,0.256332,-0.041508,...,-0.460160,-0.211042,-0.193154,-0.193572,-0.146217,0.009451,0.008404,0.008493,0.063312,0.054056
75%,1597.500000,885.750000,0.922057,0.650533,0.631683,0.869990,0.395910,0.774676,0.921263,0.820640,...,0.382191,0.274779,0.125947,0.587377,0.677040,0.569285,0.650856,0.676415,0.641446,0.599094
max,2146.000000,1215.000000,2.033851,3.300640,2.634831,2.856874,2.279619,1.951516,2.464849,2.825331,...,2.918326,7.756118,9.473201,5.085624,4.965028,2.858758,2.817239,5.674778,3.746576,5.471818


info() permet d'afficher le type de chaque colonnes et si elles sont nulles ou non
describe() permet d'afficher la somme de toutes les lignes de chaque colonne avec "count", la moyenne de la colonne avec "mean", l'écart-type de la colonne avec "std", le minimum et le maximum de chaque colonne avec "min" et "max". De plus, nous avons les 25%, les 50% et les 75% premières valeurs de chaque colonne.
